In [502]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.offline as po
import plotly.express as px
import matplotlib.pyplot as plt
from matplotlib import rcParams
import calendar
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
rcParams['figure.figsize'] = 8,5
plt.style.use('fivethirtyeight')

In [162]:
trans = pd.read_excel('transaction.xlsx')
print(trans.shape)
trans.head()

(500, 7)


,amount,a/c_number,trans_type,description,reference_number,trans_branch,datetime
0,200.0,070******357,Debited,AIRTIME/ MTN/08169327250,099MJKL222600FUo,HEAD OFFICE,2022-09-17 04:45:42
1,300.0,070******357,Debited,AIRTIME/ MTN/08168550974,099MJKL22258SUsT,HEAD OFFICE,2022-09-16 03:20:03
2,500.0,070******357,Debited,TRF/From TelRich Services/FRM GOODRICH IFE,099MJKL22258LPM0,HEAD OFFICE,2022-09-15 23:29:28
3,200.0,070******357,Debited,AIRTIME/ MTN/08169327250,099MJKL2225802x3,HEAD OFFICE,2022-09-15 05:07:19
4,300.0,070******357,Debited,AIRTIME/ MTN/08168550974,099MJKL22257SR8s,HEAD OFFICE,2022-09-15 03:08:11


There are more debit transaction than credits

In [163]:
transaction_type = trans['trans_type'].value_counts()
label = transaction_type.index
size = transaction_type.values
colors = ['lightgreen', 'aqua']

trace = go.Pie(
    labels=label,
    values=size,
    marker=dict(colors=colors),
)

data = [trace]
layout = go.Layout(title="Transaction Type Distribution")
fig = go.Figure(data=data, layout=layout)
po.iplot(fig)

We can say head office handle most of the transactions

In [164]:
transaction_branch = trans['trans_branch'].value_counts()
label = transaction_branch.index
size = transaction_branch.values
colors = ['aqua', 'lightgreen', 'lightgrey', 'khaki']

trace = go.Pie(
    labels=label,
    values=size,
    marker=dict(colors=colors)
)

data = [trace]
layout = go.Layout(title = 'Transaction Branch Distribution')
fig = go.Figure(data=data, layout=layout)
po.iplot(fig)

Credit Transactions has the highest average amount used

In [165]:
trans_type_mean = round(trans.groupby('trans_type')['amount'].mean().to_frame(), 2)
trace = go.Bar(
    x=trans_type_mean.index,
    y=trans_type_mean.amount,
    marker=dict(color=trans_type_mean.amount, 
                colorscale='Reds')
)

data = [trace]
layout = go.Layout(title = 'Average Amount Used in Transaction Type',
                   yaxis = dict(title = 'Amount in Naira'))
fig= go.Figure(data=data, layout=layout)
po.iplot(fig)

Head office has the highest mean amount of transactions.

In [209]:
trans_type_mean = round(trans.groupby('trans_branch')['amount'].mean(), 2).\
                    sort_values(ascending=False).to_frame()
trace = go.Bar(
    x=trans_type_mean.index,
    y=trans_type_mean.amount,
    marker=dict(color=trans_type_mean.amount, 
                colorscale='Reds')
)

data = [trace]
layout = go.Layout(title = 'Average Amount Handled by Each Branch',
                   yaxis = dict(title = 'Amount in Naira'))
fig= go.Figure(data=data, layout=layout)
po.iplot(fig)

Description Distribution.

It's suprising how most of the money is spent on airtime.

In [518]:
trans_copy = trans.copy()
trans_copy['description'] = np.where(trans_copy['description'].str.contains('airtime', case=False), 'Airtime_Purchase',
                                trans_copy['description'])
trans_copy['description'] = np.where(trans_copy['description'].str.contains(r'\bPOS\b|ATM'), 'POS_Transaction',
                                trans_copy['description'])
trans_copy['description'] = np.where(trans_copy['description'].str.contains(r'\bTRF\b|Transfer|TRANSFER|TFR'), 'Transfer',
                                trans_copy['description'])
trans_copy['description'] = np.where(trans_copy['description'].str.contains('USSD'), 'USSD_Transaction',
                                trans_copy['description'])
trans_copy['description'] = np.where(~(trans_copy['description'] == 'Airtime_Purchase') & ~(trans_copy['description'] == 'POS_Transaction') & ~(trans_copy['description'] == 'Transfer') &\
                                        ~(trans_copy['description'] == 'USSD_Transaction'), 'Other Transaction', trans_copy['description'])
df = trans_copy['description'].value_counts()

label = df.index
size = df.values
colors = ['aqua', 'lightgreen']

trace = go.Pie(
    labels=label,
    values=size,
    marker=dict(colors=colors)
)

data = [trace]
layout = go.Layout(title='Alert Description')
fig = go.Figure(data=data, layout=layout)
po.iplot(fig)


Central Processing Branch handle only POS transaction, more like web transaction.

In [519]:
Pos = trans_copy[trans_copy['description'] == 'POS_Transaction']
list(Pos['trans_branch'].unique())

['CENTRAL PROCESSING BRANCH', 'HEAD OFFICE']

Monthly Cash Flow for 2022

In [520]:
trans_copy = trans_copy.set_index('datetime')
grouped_trans = trans_copy.groupby(by=[trans_copy.index.year, trans_copy.index.month])['amount'].sum().to_frame().\
                    reset_index(level=1).rename(columns={'datetime':'month'}).reset_index().\
                        rename(columns={'datetime':'year'})
                        
yr_2022 = grouped_trans[1:]
yr_2022['month'] = yr_2022['month'].apply(lambda x: calendar.month_abbr[x])
yr_2022

,year,month,amount
1,2022,Jan,206475.13
2,2022,Feb,117178.63
3,2022,Mar,116000.13
4,2022,Apr,118605.00
5,2022,May,91240.63
6,2022,Jun,84723.38
7,2022,Jul,107131.88
8,2022,Aug,529029.46
9,2022,Sep,51721.50


In [521]:
trace = px.bar(yr_2022,
    x='month',
    y='amount',
    text_auto = '.2s',
    labels={'month':'Month', 'amount':'Amount in Naira'},
    title='Monthly Cash Flow'
)
trace.update_traces(textposition='outside', cliponaxis=False)
trace

Cash Flow Breakdown

In [522]:
monthly_trans_typ = trans_copy.groupby(by=[trans_copy.index.year, trans_copy.index.month, 'trans_type'])['amount'].sum().to_frame()
monthly_trans_typ.reset_index(level=[1,2], inplace=True)
monthly_trans_typ.rename(columns={'datetime':'month'}, inplace=True)
monthly_trans_typ.reset_index(inplace=True)
monthly_trans_typ.rename(columns={'datetime':'year'}, inplace=True)
monthly_trans_typ.set_index('year', inplace=True)
monthly_trans_typ['month'] = monthly_trans_typ['month'].apply(lambda x: calendar.month_abbr[x])
yr_2022 = monthly_trans_typ[2:]
yr_2022

,month,trans_type,amount
year,,,
2022,Jan,Credited,96250.00
2022,Jan,Debited,110225.13
2022,Feb,Credited,56376.00
2022,Feb,Debited,60802.63
2022,Mar,Credited,58600.00
2022,Mar,Debited,57400.13
2022,Apr,Credited,59300.00
2022,Apr,Debited,59305.00
2022,May,Credited,46662.50


In [523]:
trace = px.bar(yr_2022,
    x='month',
    y='amount',
    color='trans_type',
    barmode='group',
    text_auto='.2s',
    title='Cash Flow Breakdown',
    labels={'month':'Month', 'amount':'Amount in Naira'},
)
trace.update_traces(textposition='outside', cliponaxis=False, textfont_size = 10)

In [525]:
fig = px.sunburst(monthly_trans_typ, path=[monthly_trans_typ.index, 'month', 'trans_type'], values='amount')
fig

In [526]:
fig = px.sunburst(trans_copy, path=['trans_branch', 'description', 'trans_type'], values='amount')
fig